# 0.1. imports

In [1]:
# lca
import bw2data
import bw2io
# i/o
import gzip
# data science
import pandas as pd
import  numpy
import scipy

In [2]:
%load_ext autoreload
%autoreload 2

# 1. load required data
## 1.1. load `pylcaio` object

In [3]:
with gzip.open('/Users/michaelweinold/data/data_processed/hybrid.pickle','rb') as f:
    hybrid_pickle = pd.read_pickle(f)

# 2. refactoring
## 2.1. legacy function

In [ ]:
def import_exiobase_into_brightway2(self):

        print('Starting import of exiobase')
        print('Aggregating results')
        # calculating aggregated results for exiobase
        self.Lio = scipy.sparse.csr_matrix(
            np.linalg.solve(np.eye(self.A_io.shape[0]) - self.A_io.todense(), np.eye(self.A_io.shape[0])))
        aggregated_results = pd.DataFrame((self.C_io.dot(self.F_io).dot(self.Lio)).todense(), self.IO_impact_categories,
                                          pd.MultiIndex.from_product([self.regions_of_IO, self.sectors_of_IO]))
        print('Formating data')
        # create hash codes for exiobase sectors
        give_hash_to_exio = dict.fromkeys(aggregated_results.columns)
        give_hash_to_exio = {k: hashlib.md5(str(k).encode()).hexdigest() for k, v in give_hash_to_exio.items()}
        aggregated_results.columns = [give_hash_to_exio[i] for i in aggregated_results.columns]

        # link IMPACT World+ impact categories typology to the methods abbreviations used in brightway2
        IW_pylcaio_to_bw2 = dict.fromkeys(self.IO_impact_categories)
        for IW_category in IW_pylcaio_to_bw2:
            if 'PDF' in IW_category or 'DALY' in IW_category:
                IW_pylcaio_to_bw2[IW_category] = \
                bw2data.methods.get( ('IMPACTWorld+ (Default_Recommended_Damage 1.46)', IW_category[0].split(' (')[0]) )[
                    'abbreviation']
            else:
                IW_pylcaio_to_bw2[IW_category] = \
                bw2data.methods.get(('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)', IW_category.split(' (')[0]))[
                    'abbreviation']
        aggregated_results.index = list(IW_pylcaio_to_bw2.values())

## 2.2. investigation

In [38]:
bw2data.methods

Methods dictionary with 975 objects, including:
	('CML 2001 (superseded)', 'acidification potential', 'average European')
	('CML 2001 (superseded)', 'acidification potential', 'generic')
	('CML 2001 (superseded)', 'climate change', 'GWP 100a')
	('CML 2001 (superseded)', 'climate change', 'GWP 20a')
	('CML 2001 (superseded)', 'climate change', 'GWP 500a')
	('CML 2001 (superseded)', 'climate change', 'lower limit of net GWP')
	('CML 2001 (superseded)', 'climate change', 'upper limit of net GWP')
	('CML 2001 (superseded)', 'eutrophication potential', 'average European')
	('CML 2001 (superseded)', 'eutrophication potential', 'generic')
	('CML 2001 (superseded)', 'freshwater aquatic ecotoxicity', 'FAETP 100a')
Use `list(this object)` to get the complete list.

In [86]:
list_IO_impact_categories

[('Problem oriented approach: baseline (CML, 2001)',
  'abiotic depletion (elements, ultimate ultimate reserves)',
  'ADPelements (Oers et al. 2001)',
  'kg antimony eq.'),
 ('Problem oriented approach: baseline (CML, 2001)',
  'abiotic depletion (fossil fuels)',
  'ADPfossil fuels (Oers et al., 2001)',
  'MJ'),
 ('Problem oriented approach: non baseline (CML, 2001)',
  'abiotic depletion (elements, reserve base)',
  'ADPelements (Oers et al. 2001)',
  'kg antimony eq.'),
 ('Problem oriented approach: non baseline (CML, 2001)',
  'abiotic depletion (elements, economic reserve)',
  'ADPelements (Oers et al. 2001)',
  'kg antimony eq.'),
 ('Problem oriented approach: non baseline (CML, 2001)',
  'Landuse increase of land competition',
  'LUC (Guinee et al, 2001)',
  'm2*yr'),
 ('Problem oriented approach: baseline (CML, 2001)',
  'global warming (GWP100)',
  'GWP100 (IPCC, 2007)',
  'kg CO2 eq.'),
 ('Problem oriented approach: non baseline (CML, 2001)',
  'global warming net (GWP100 min)

In [41]:
tuple_test = ('IMPACTWorld+ (Default_Recommended_Damage 1.46)', 'Human toxicity cancer, short term')

In [42]:
[*bw2data.methods][-20:-1]

[('selected LCI results', 'air', 'NMVOC'),
 ('selected LCI results', 'air', 'sulphur dioxide'),
 ('selected LCI results', 'air', 'CO2, fossil'),
 ('TRACI', 'human health', 'non-carcinogenics'),
 ('TRACI', 'human health', 'respiratory effects, average'),
 ('TRACI', 'human health', 'carcinogenics'),
 ('TRACI', 'environmental impact', 'eutrophication'),
 ('TRACI', 'environmental impact', 'global warming'),
 ('TRACI', 'environmental impact', 'ecotoxicity'),
 ('TRACI', 'environmental impact', 'photochemical oxidation'),
 ('TRACI', 'environmental impact', 'acidification'),
 ('TRACI', 'environmental impact', 'ozone depletion'),
 ('USEtox w/o LT', 'human toxicity w/o LT', 'carcinogenic w/o LT'),
 ('USEtox w/o LT', 'human toxicity w/o LT', 'non-carcinogenic w/o LT'),
 ('USEtox w/o LT', 'ecotoxicity w/o LT', 'total w/o LT'),
 ('USEtox w/o LT', 'human toxicity w/o LT', 'total w/o LT'),
 ('USEtox', 'human toxicity', 'non-carcinogenic'),
 ('USEtox', 'human toxicity', 'carcinogenic'),
 ('USEtox', 'e

In [25]:
[*bw2data.methods.keys()][0:5]

[('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP infinite'),
 ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP 500a'),
 ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP 20a'),
 ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP 100a'),
 ('CML 2001 (superseded)', 'malodours air', 'malodours air')]

In [32]:
list_tuples_LCA_impact_categories = [*bw2data.methods.keys()]

In [43]:
list_tuples_LCA_impact_categories[10]

('CML 2001 (superseded)', 'climate change', 'lower limit of net GWP')

In [44]:
[item for item in list_tuples_LCA_impact_categories if 'CML' in item[0]][5]

('CML 2001 (superseded)', 'human toxicity', 'HTP 20a')

In [26]:
tuple1 = ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP infinite')

In [27]:
bw2data.methods.get(tuple1)

{'description': '',
 'filename': 'LCIA_Implementation_3.8.xlsx',
 'unit': 'kg 1,4-DCB-Eq',
 'abbreviation': 'cml-2001-supersededtt.eef0ae2524095b1311b8bd8c6d437e20',
 'num_cfs': 581}

In [6]:
type(hybrid_pickle)

dict

In [8]:
[*hybrid_pickle.keys()]

['PRO_f',
 'A_ff',
 'A_io',
 'A_io_f',
 'F_f',
 'F_io',
 'C_f',
 'C_io',
 'hybridized_processes',
 'description',
 'sectors_of_IO',
 'regions_of_IO',
 'flows_of_IO',
 'impact_categories_IO',
 'IMP',
 'STR']

In [ ]:
IW_pylcaio_to_bw2 = dict.fromkeys(self.impact_categories_IO)
for IW_category in IW_pylcaio_to_bw2:
    if 'PDF' in IW_category or 'DALY' in IW_category:
        IW_pylcaio_to_bw2[IW_category] = \
        bw2data.methods.get(('IMPACTWorld+ (Default_Recommended_Damage 1.46)', IW_category.split(' (')[0]))[
            'abbreviation']

In [22]:
type(bw2data.methods.get(('IMPACTWorld+ (Default_Recommended_Damage 1.46)', 'Human toxicity (cancer)')))

NoneType

In [45]:
list_IO_impact_categories: list = hybrid_pickle['impact_categories_IO']

In [63]:
list_IO_impact_categories[0][0].split(' (')[0]

'Problem oriented approach: baseline'

In [59]:
[str(item).split(' (')[0] for item in list_IO_impact_categories]

["('Problem oriented approach: baseline",
 "('Problem oriented approach: baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: non baseline",
 "('Problem oriented approach: baseline",
 "('Problem oriented approach: baseline",
 "('Problem oriented approach: baseline",
 "('Problem oriented approach: non baseline",
 "('Problem orie

In [58]:
list_IO_impact_categories[-200:-1]

[('Problem oriented approach: baseline (CML, 2001)',
  'abiotic depletion (elements, ultimate ultimate reserves)',
  'ADPelements (Oers et al. 2001)',
  'kg antimony eq.'),
 ('Problem oriented approach: baseline (CML, 2001)',
  'abiotic depletion (fossil fuels)',
  'ADPfossil fuels (Oers et al., 2001)',
  'MJ'),
 ('Problem oriented approach: non baseline (CML, 2001)',
  'abiotic depletion (elements, reserve base)',
  'ADPelements (Oers et al. 2001)',
  'kg antimony eq.'),
 ('Problem oriented approach: non baseline (CML, 2001)',
  'abiotic depletion (elements, economic reserve)',
  'ADPelements (Oers et al. 2001)',
  'kg antimony eq.'),
 ('Problem oriented approach: non baseline (CML, 2001)',
  'Landuse increase of land competition',
  'LUC (Guinee et al, 2001)',
  'm2*yr'),
 ('Problem oriented approach: baseline (CML, 2001)',
  'global warming (GWP100)',
  'GWP100 (IPCC, 2007)',
  'kg CO2 eq.'),
 ('Problem oriented approach: non baseline (CML, 2001)',
  'global warming net (GWP100 min)

In [55]:
[item for item in list_IO_impact_categories if 'antimony' in str(item)]

[('Problem oriented approach: baseline (CML, 2001)',
  'abiotic depletion (elements, ultimate ultimate reserves)',
  'ADPelements (Oers et al. 2001)',
  'kg antimony eq.'),
 ('Problem oriented approach: non baseline (CML, 2001)',
  'abiotic depletion (elements, reserve base)',
  'ADPelements (Oers et al. 2001)',
  'kg antimony eq.'),
 ('Problem oriented approach: non baseline (CML, 2001)',
  'abiotic depletion (elements, economic reserve)',
  'ADPelements (Oers et al. 2001)',
  'kg antimony eq.')]

In [72]:
list_bw2data_methods = pd.read_pickle('/Users/michaelweinold/github/bw_hybrid/notebooks/list_bw2data_methods.pickle')

In [76]:
[item for item in list_bw2data_methods if '99' in str(item)]

[('eco-indicator 99, (E,E) (superseded)',
  'ecosystem quality',
  'acidification & eutrophication'),
 ('eco-indicator 99, (E,E) (superseded)',
  'ecosystem quality',
  'land occupation'),
 ('eco-indicator 99, (E,E) (superseded)', 'ecosystem quality', 'ecotoxicity'),
 ('eco-indicator 99, (E,E) (superseded)', 'human health', 'climate change'),
 ('eco-indicator 99, (E,E) (superseded)',
  'human health',
  'respiratory effects'),
 ('eco-indicator 99, (E,E) (superseded)',
  'human health',
  'ozone layer depletion'),
 ('eco-indicator 99, (E,E) (superseded)',
  'human health',
  'ionising radiation'),
 ('eco-indicator 99, (E,E) (superseded)', 'human health', 'carcinogenics'),
 ('eco-indicator 99, (E,E) (superseded)', 'resources', 'mineral extraction'),
 ('eco-indicator 99, (E,E) (superseded)', 'resources', 'fossil fuels'),
 ('eco-indicator 99, (E,E) (superseded)', 'resources', 'total'),
 ('eco-indicator 99, (E,E) (superseded)', 'human health', 'total'),
 ('eco-indicator 99, (E,E) (supersede

In [81]:
[item for item in list_bw2data_methods if 'IMPACTWorld+' in str(item)]

[('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)',
  'Climate change, short term'),
 ('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)',
  'Climate change, long term'),
 ('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)',
  'Fossil and nuclear energy use'),
 ('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)', 'Mineral resources use'),
 ('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)',
  'Photochemical oxidant formation'),
 ('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)', 'Ozone Layer Depletion'),
 ('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)',
  'Freshwater ecotoxicity'),
 ('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)', 'Human toxicity cancer'),
 ('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)',
  'Human toxicity non cancer'),
 ('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)', 'Water scarcity'),
 ('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)',
  'Freshwater acidification'),
 ('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)',
  'Terrestrial acidifica

In [84]:
[item for item in list_IO_impact_categories if 'acidification' in str(item)]

[('Problem oriented approach: baseline (CML, 2001)',
  'acidification (incl. fate, average Europe total, A&B)',
  'AP ( Huijbregts, 1999; average Europe total, A&B)',
  'kg SO2 eq.'),
 ('Problem oriented approach: non baseline (CML, 2001)',
  'acidification (fate not incl.)',
  'AP (Hauschild & Wenzel (1998).',
  'kg SO2 eq.'),
 ('ECOINDICATOR 99 (H.A)',
  'Damage to Ecosystem Quality caused by the combined effect of acidification and eutrophication (H.A)',
  'Damage to Ecosystem Quality caused by the combined effect of acidification and eutrophication (H.A)',
  'PDF*m2*yr'),
 ('ECOINDICATOR 99 (E.E)',
  'Damage to Ecosystem Quality caused by the combined effect of acidification and eutrophication (E.E)',
  'Damage to Ecosystem Quality caused by the combined effect of acidification and eutrophication (E.E)',
  'PDF*m2*yr'),
 ('ECOINDICATOR 99 (I.I)',
  'Damage to Ecosystem Quality caused by the combined effect of acidification and eutrophication (I.I)',
  'Damage to Ecosystem Quality c

In [79]:
bw2data.methods.get(tuple(['IMPACTWorld+ (Default_Recommended_Midpoint 1.28)', 'Problem oriented approach: baseline']))

In [85]:
list_IO_impact_categories: list = hybrid_pickle['impact_categories_IO']

dict_matching_impact_categories: dict = dict.fromkeys(list_IO_impact_categories, None),

for IO_impact_category in list_IO_impact_categories:
    #print(IO_impact_category)
    retvar = [item for item in list_bw2data_methods if IO_impact_category[0].split(' (')[0] in str(item)]
    split_result: str = IO_impact_category[1].split(' (')[0]
    #print(split_result)
    #dict_matching_impact_categories[IO_impact_category] = LCA_impact_category

In [75]:
list_IO_impact_categories: list = hybrid_pickle['impact_categories_IO']

list_match_1: list = ['PDF', 'DALY']

dict_matching_impact_categories: dict = dict.fromkeys(list_IO_impact_categories, None),

for IO_impact_category in list_IO_impact_categories:
    if any(substring in IO_impact_category for substring in list_match_1):
        #print(IO_impact_category)
        split_result: str = IO_impact_category[0].split(' (')[0]
        print(split_result)
        #dict_matching_impact_categories[IO_impact_category] = LCA_impact_category

ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99
ECOINDICATOR 99


## 2.3. new function
